# Домашнее задание


# 🎈🍾 🎉





Вы работаете в крупном онлайн-магазине, который реализует товары «всё для праздников». Вам необходимо провести ABC-анализ и выявить наиболее важные товары на основе объемов продаж и популярности. Эта информация составит план закупок на весь следующий год. 

**ABC-анализ** позволяет определить наиболее важные ресурсы компании на основе объемов продаж и прибыли.

В основе ABC-анализа лежит __«Принцип Парето»: 20% ресурсов приносят 80% прибыли__. Именно этот вид анализа позволяет выделить те самые  группы товаров, ресурсов, клиентов, поставщиков и т. п., которые и приносят компании основную прибыль.

__Задача анализа разделить товары на группы по принципу:__

- А — наиболее ценные, 20% — ассортимента (номенклатура); 80% — продаж
- В — промежуточные, 30% — ассортимента; 15% — продаж
- С — наименее ценные, 50% — ассортимента; 5% — продаж

## Задание 1

Сохраните все данные из таблицы `products` из PostgreSQL в дата-фрейм `products` и подготовьте данные к анализу.

1. Подгрузите данные для подключения
2. Подключите нужную библиотеку для работы с PostgreSQL
3. Создайте соединение с БД
4. Создайте запрос и выгрузите все данные из таблицы `products`
5. Положите данные в дата-фрейм `products`
6. Приведите данные к удобному типу: числа к числовому типу, идентификаторы — к текстовому


In [536]:
from postgresql_secret import pg_secret

In [537]:
import psycopg2

connection = psycopg2.connect(
                             database=pg_secret['db_name'],
                             user=pg_secret['db_user'],
                             password=pg_secret['db_password'],
                             host=pg_secret['db_host'],
                             port=pg_secret['db_port'],
                             )
# создай соединение

cursor = connection.cursor()
# заведи курсор, чтобы делать запросы в базу

In [538]:
import pandas as pd

query = "SELECT * FROM products"
cursor.execute(query)            
result = cursor.fetchall()
# result

columns = []                 
for desc in cursor.description:  
    columns.append(desc[0]) 


products = pd.DataFrame(result, columns=columns)  # Из списка кортежей сделаем датафрейм
products.head()

,invoice_id,stock_code,description,quantity,price
0,493955,21894,POTTING SHED SEED ENVELOPES,12,1.25
1,493955,85113,POLKA DOT PISTACHIO PAIL,6,2.10
2,493955,15056BL,EDWARDIAN PARASOL BLACK,3,5.95
3,493955,20679,EDWARDIAN PARASOL RED,3,5.95
4,493955,85014B,RED/WHITE DOTS RUFFLED UMBRELLA,3,5.95


## Задание 2

__Посчитайте, какой доход принес каждый продукт и сколько единиц каждого продукта было продано__

1. Посчитай общую сумму оплат для каждой строки счета
2. Получи общий доход и количество единиц по товарам
3. Объедини сгруппированные данные в один датафрейм со столбцами: `description`, `stock_code`, `quantity`, `total`

In [540]:
products['amount'] = products.quantity * products.price

products['total'] = products.apply('amount', axis=1)
products.head()

,invoice_id,stock_code,description,quantity,price,amount,total
0,493955,21894,POTTING SHED SEED ENVELOPES,12,1.25,15.00,15.00
1,493955,85113,POLKA DOT PISTACHIO PAIL,6,2.10,12.60,12.60
2,493955,15056BL,EDWARDIAN PARASOL BLACK,3,5.95,17.85,17.85
3,493955,20679,EDWARDIAN PARASOL RED,3,5.95,17.85,17.85
4,493955,85014B,RED/WHITE DOTS RUFFLED UMBRELLA,3,5.95,17.85,17.85


In [541]:
# посчитай общих доход по каждому товару
products_total_amount = products.groupby('stock_code')['total'].sum().reset_index()
products_total_amount.head()

,stock_code,total
0,10002,6638.27
1,10080,124.61
2,10109,1.68
3,10120,136.08
4,10123C,226.76


In [542]:
# посчитай общее количество проданных единиц по каждому товару
products_product_amount = products.groupby('stock_code')['quantity'].sum().reset_index()
products_product_amount.head()

,stock_code,quantity
0,10002,8479
1,10080,303
2,10109,4
3,10120,648
4,10123C,628


In [543]:
# объедини общие данные в одну таблицу
prod = pd.merge(products_total_amount,
                    products_product_amount,
                    on='stock_code',
                    how='inner')
prod.head()

,stock_code,total,quantity
0,10002,6638.27,8479
1,10080,124.61,303
2,10109,1.68,4
3,10120,136.08,648
4,10123C,226.76,628


## Задание 3

__Найди нарастающий итог по процентам__

1. Отсортируй датафрейм по доходу в нисходящем порядке 
2. Найди процент дохода каждого товара в общем доходе 
3. Сделай нарастающий итог (running total) по процентам каждого товара, для этого найди в интернете и примени метод для расчета кумулятивной суммы в `pandas`. 


Нарастающий итог — это представление данных, в котором мы видим динамику нарастания или убывания суммы от начала отсчета до конца. Получаем мы эту кумулятивную сумму, прибавлением поочередно каждого следующего элемента к сумме предыдущих.  



In [544]:
# отсортируй дата-фрейм (большие вверху, маленькие внизу)
prod = prod.sort_values(['total'],ascending=False)
prod

,stock_code,total,quantity
1609,22423,277656.25,24139
4080,85123A,247203.36,91814
2726,23843,168469.60,80995
4058,85099B,167920.64,93436
4625,M,151777.67,9391
...,...,...,...
4630,TEST002,1.00,1
247,20721,0.85,1
3419,84205C,0.38,2
2847,35930,0.38,1


In [546]:
# найди процент дохода для каждого товара 

#(искомая часть / целое число) * 100

#целое число
prod['total_sum'] = prod['total'].sum()
prod['total_sum'] = prod.apply('total_sum', axis=1)

#процент дохода для каждого товара 
prod['percent_per_product'] = ((prod.total / prod.total_sum) * 100) 

#добавление столбца с процентом
prod['percent_per_product'] = prod.apply('percent_per_product', axis=1)

#100% (проверка)
# prod.total_percent = prod['percent_per_product'].sum()

prod

,stock_code,total,quantity,total_sum,percent_per_product
1609,22423,277656.25,24139,17374804.25,1.598039586546708864360299196
4080,85123A,247203.36,91814,17374804.25,1.422769180262851018882701945
2726,23843,168469.60,80995,17374804.25,0.9696201325548746829766441829
4058,85099B,167920.64,93436,17374804.25,0.9664606149447698094210183692
4625,M,151777.67,9391,17374804.25,0.8735503883446629333967892041
...,...,...,...,...,...
4630,TEST002,1.00,1,17374804.25,0.000005755460525548079196345478252
247,20721,0.85,1,17374804.25,0.000004892141446715867316893656514
3419,84205C,0.38,2,17374804.25,0.000002187074999708270094611281736
2847,35930,0.38,1,17374804.25,0.000002187074999708270094611281736


In [547]:
# используй метод pandas для кумулятивной суммы

#1 без добавленного столбца
# print(prod.cumsum(axis = 0))

#2 с добавленным столбцом
prod['running_total'] = prod['percent_per_product'].cumsum(axis = 0)
prod['running_total'] = prod.apply('running_total', axis=1)
prod

,stock_code,total,quantity,total_sum,percent_per_product,running_total
1609,22423,277656.25,24139,17374804.25,1.598039586546708864360299196,1.598039586546708864360299196
4080,85123A,247203.36,91814,17374804.25,1.422769180262851018882701945,3.020808766809559883243001141
2726,23843,168469.60,80995,17374804.25,0.9696201325548746829766441829,3.990428899364434566219645324
4058,85099B,167920.64,93436,17374804.25,0.9664606149447698094210183692,4.956889514309204375640663693
4625,M,151777.67,9391,17374804.25,0.8735503883446629333967892041,5.830439902653867309037452897
...,...,...,...,...,...,...
4630,TEST002,1.00,1,17374804.25,0.000005755460525548079196345478252,99.99999073370855386759249359
247,20721,0.85,1,17374804.25,0.000004892141446715867316893656514,99.99999562585000058345981048
3419,84205C,0.38,2,17374804.25,0.000002187074999708270094611281736,99.99999781292500029172990509
2847,35930,0.38,1,17374804.25,0.000002187074999708270094611281736,99.99999999999999999999999970


## Задание 4

__Пометь товары метками `'A'`, `'B'`, `'C'` в зависимости от нарастающего итога__

1. Напиши функцию, которая будет помечать товары с нарастающим итогом ниже или равно 80% как группу `'A'`, от 80%-95% — как `'B'` и остальные как `'C'`

In [548]:
prod['running_total'].describe()

count                              4631
unique                             4630
top       99.99999999999999999999999970
freq                                  2
Name: running_total, dtype: object

In [549]:
prod['running_total'] = prod['running_total'].apply(float)

In [550]:
def revenue(x):
    if x <= 80:
        return 'A'
    elif x <= 95:
        return 'B'
    else:
        return 'C'

# создай новый столбец revenue с помощью функции
prod['revenue'] = prod['running_total'].apply(revenue)
prod

,stock_code,total,quantity,total_sum,percent_per_product,running_total,revenue
1609,22423,277656.25,24139,17374804.25,1.598039586546708864360299196,1.598040,A
4080,85123A,247203.36,91814,17374804.25,1.422769180262851018882701945,3.020809,A
2726,23843,168469.60,80995,17374804.25,0.9696201325548746829766441829,3.990429,A
4058,85099B,167920.64,93436,17374804.25,0.9664606149447698094210183692,4.956890,A
4625,M,151777.67,9391,17374804.25,0.8735503883446629333967892041,5.830440,A
...,...,...,...,...,...,...,...
4630,TEST002,1.00,1,17374804.25,0.000005755460525548079196345478252,99.999991,C
247,20721,0.85,1,17374804.25,0.000004892141446715867316893656514,99.999996,C
3419,84205C,0.38,2,17374804.25,0.000002187074999708270094611281736,99.999998,C
2847,35930,0.38,1,17374804.25,0.000002187074999708270094611281736,100.000000,C


## Задание 5

__Найди нарастающий итог по количеству товарных единиц и пометь товары метками `'A'`, `'B'`, `'C'`, отмечая тем самым популярность товара__

1. Отсортируй предыдущий датафрейм с данными о доходе теперь по количеству товаров
2. Найди процент каждого товара в общем количестве
3. Найди нарастающий итог по количеству
4. Напиши функцию с теми же параметрами, что и у дохода
5. Отметь группу

In [551]:
# проведи те же операции, что в двух предыдущих заданиях, для оценки популярности товаров

#сортировка по количеству товаров
prod = prod.sort_values(['quantity'],ascending=False)
# prod.head()

In [552]:
# найди процент каждого товара в общем количестве ((искомая часть / целое число) * 100)
#целое число
prod['total_quantity'] = prod['quantity'].sum()
prod['total_quantity'] = prod.apply('total_quantity', axis=1)

#процент дохода для каждого товара 
prod['percent_per_quantity'] = ((prod.quantity / prod.total_quantity) * 100) 

#добавление столбца с процентом
prod['percent_per_quantity'] = prod.apply('percent_per_quantity', axis=1)
# prod.head()

In [553]:
# нарастающий итог по количеству
prod['running_quantity'] = prod['percent_per_quantity'].cumsum(axis = 0)
prod['running_quantity'] = prod.apply('running_quantity', axis=1)
# prod.head()

In [554]:
# prod['running_total'].describe()

In [555]:
def quantity(x):
    if x <= 80:
        return 'A'
    elif x <= 95:
        return 'B'
    else:
        return 'C'
    
# создай новый столбец popularity с помощью функции
prod['popularity'] = prod['running_quantity'].apply(quantity)
# prod

In [556]:
# prod.head()
prod.head()

,stock_code,total,quantity,total_sum,percent_per_product,running_total,revenue,total_quantity,percent_per_quantity,running_quantity,popularity
3410,84077,24098.03,105185,17374804.25,0.1386952604084733789159092253,34.446876,A,10528705,0.999031,0.999031,A
4058,85099B,167920.64,93436,17374804.25,0.9664606149447698094210183692,4.956890,A,10528705,0.887441,1.886471,A
4080,85123A,247203.36,91814,17374804.25,1.422769180262851018882701945,3.020809,A,10528705,0.872035,2.758506,A
601,21212,43390.03,89850,17374804.25,0.2497296048673469227718061917,18.467322,A,10528705,0.853381,3.611888,A
2726,23843,168469.60,80995,17374804.25,0.9696201325548746829766441829,3.990429,A,10528705,0.769278,4.381166,A


## Задание 6

__Объедини группы и проанализируй__

1. Создай серию `group` с объединенным столбцом отметок популярности и доходности товаров
2. Посчитай количество вхождений каждой сгруппированной пары
3. Какие группы непопулярны и приносят мало дохода?
4. На каких группах товаров нужно сфокусироваться при составлении плана закупок?

In [557]:
prod_1 = prod[['stock_code', 'total', 'quantity', 'percent_per_product', 'percent_per_quantity', 'running_total', 'running_quantity', 'revenue', 'popularity']]
prod_1

,stock_code,total,quantity,percent_per_product,percent_per_quantity,running_total,running_quantity,revenue,popularity
3410,84077,24098.03,105185,0.1386952604084733789159092253,0.999031,34.446876,0.999031,A,A
4058,85099B,167920.64,93436,0.9664606149447698094210183692,0.887441,4.956890,1.886471,A,A
4080,85123A,247203.36,91814,1.422769180262851018882701945,0.872035,3.020809,2.758506,A,A
601,21212,43390.03,89850,0.2497296048673469227718061917,0.853381,18.467322,3.611888,A,A
2726,23843,168469.60,80995,0.9696201325548746829766441829,0.769278,3.990429,4.381166,A,A
...,...,...,...,...,...,...,...,...,...
2907,37438,3.75,1,0.00002158297697080529698629554344,0.000009,99.999388,99.999962,C,C
4279,90025B,3.75,1,0.00002158297697080529698629554344,0.000009,99.999345,99.999972,C,C
4241,90004A,3.75,1,0.00002158297697080529698629554344,0.000009,99.999324,99.999981,C,C
2874,35974,3.75,1,0.00002158297697080529698629554344,0.000009,99.999280,99.999991,C,C


In [558]:
#cерия group с объединенным столбцом отметок популярности и доходности товаров
#prod_1['group_RP'] = prod_1['revenue'] + prod_1['popularity']
group_RP_column = prod_1['revenue'] + prod_1['popularity']
prod_1 = prod_1.assign(group_RP=group_RP_column)
prod_1

,stock_code,total,quantity,percent_per_product,percent_per_quantity,running_total,running_quantity,revenue,popularity,group_RP
3410,84077,24098.03,105185,0.1386952604084733789159092253,0.999031,34.446876,0.999031,A,A,AA
4058,85099B,167920.64,93436,0.9664606149447698094210183692,0.887441,4.956890,1.886471,A,A,AA
4080,85123A,247203.36,91814,1.422769180262851018882701945,0.872035,3.020809,2.758506,A,A,AA
601,21212,43390.03,89850,0.2497296048673469227718061917,0.853381,18.467322,3.611888,A,A,AA
2726,23843,168469.60,80995,0.9696201325548746829766441829,0.769278,3.990429,4.381166,A,A,AA
...,...,...,...,...,...,...,...,...,...,...
2907,37438,3.75,1,0.00002158297697080529698629554344,0.000009,99.999388,99.999962,C,C,CC
4279,90025B,3.75,1,0.00002158297697080529698629554344,0.000009,99.999345,99.999972,C,C,CC
4241,90004A,3.75,1,0.00002158297697080529698629554344,0.000009,99.999324,99.999981,C,C,CC
2874,35974,3.75,1,0.00002158297697080529698629554344,0.000009,99.999280,99.999991,C,C,CC


In [559]:
prod_2 = prod_1[['stock_code', 'total', 'quantity', 'running_total', 'running_quantity', 'group_RP']]
prod_2

,stock_code,total,quantity,running_total,running_quantity,group_RP
3410,84077,24098.03,105185,34.446876,0.999031,AA
4058,85099B,167920.64,93436,4.956890,1.886471,AA
4080,85123A,247203.36,91814,3.020809,2.758506,AA
601,21212,43390.03,89850,18.467322,3.611888,AA
2726,23843,168469.60,80995,3.990429,4.381166,AA
...,...,...,...,...,...,...
2907,37438,3.75,1,99.999388,99.999962,CC
4279,90025B,3.75,1,99.999345,99.999972,CC
4241,90004A,3.75,1,99.999324,99.999981,CC
2874,35974,3.75,1,99.999280,99.999991,CC


In [560]:
#Посчитай количество вхождений каждой сгруппированной пары
prod_2['group_RP'].value_counts()

CC    2041
AA     685
BB     577
BC     377
CB     303
BA     270
AB     248
CA      72
AC      58
Name: group_RP, dtype: int64

Какие группы непопулярны и приносят мало дохода? На каких группах товаров нужно сфокусироваться при составлении плана закупок?

Группы АС и СА непопулярны и приносят мало дохода.
При составления плана закупок необходимо сфокусироваться на группах СС, АА, ВВ. В слудующую очередь необходимо рассмотреть также и группы ВС, СВ, ВА и АВ.

## Задание 7

__Выгрузи данные по товарам с метокой по ABC-анализу__

1. Объедини по `stock_code` в одной таблице `ABC-код`, описание товара и `ID `

2. Выгрузи таблицу в `csv` для отправки в отдел закупок

In [561]:
ABC_analysis = pd.merge(prod_2,
                    products,
                    on='stock_code',
                    how='inner')

ABC_analysis_final = ABC_analysis[['stock_code', 'group_RP', 'description', 'invoice_id']]
ABC_analysis_final

,stock_code,group_RP,description,invoice_id
0,84077,AA,WORLD WAR 2 GLIDERS ASSTD DESIGNS,494233
1,84077,AA,WORLD WAR 2 GLIDERS ASSTD DESIGNS,494634
2,84077,AA,WORLD WAR 2 GLIDERS ASSTD DESIGNS,494666
3,84077,AA,WORLD WAR 2 GLIDERS ASSTD DESIGNS,495059
4,84077,AA,WORLD WAR 2 GLIDERS ASSTD DESIGNS,495237
...,...,...,...,...
779490,37438,CC,FUNKY FLOWER COFFEE POT,491697
779491,90025B,CC,BAROQUE BUTTERFLY EARRINGS RED,500356
779492,90004A,CC,JADE GREEN PAIR ENAMEL HAIR SLIDES,492075
779493,35974,CC,S/6 CHRISTMAS TREE PLACE SETTING,492423


In [562]:
#Выгрузка таблицы в csv

ABC_analysis_final.to_csv('ABC_analysis_final.csv', index=False)